In [3]:
# Library Package
import pandas as pd
import numpy as np
import re

# Folder Path
files = './training/training.csv'

# Regular Expression
regex_wage = r'(\$[0-9]{2,}[kK]?)'


In [34]:
frame = pd.read_csv(files, sep='|')
frame.head()

,content,industries,location,title,type_empl,url,wage
0,Amgen is committed to unlocking the potential ...,Healthcare & Medical,"Sydney, Ryde & Macquarie Park","Transformation, Senior Manager - Data, Analyti...",Full Time,https://www.seek.com.au/job/38884116?type=prom...,NaN
1,"Our client, a respected and leading global ban...",Science & Technology,"Sydney, CBD, Inner West & Eastern Suburbs",Senior Analyst | Customer Insights | Banking J...,Full Time,https://www.seek.com.au/job/38983503?type=stan...,NaN
2,We are looking for an experienced Senior Busin...,Information & Communication Technology,"Sydney, North Shore & Northern Beaches","Senior Business Analyst, Data & Analytics (Cor...",Contract/Temp,https://www.seek.com.au/job/38980384?type=stan...,NaN
3,Senior Data Analysts - check this out! This is...,"Advertising, Arts & Media","Sydney, CBD, Inner West & Eastern Suburbs",Data and Analytics Director Job in Sydney - SEEK,Full Time,https://www.seek.com.au/job/39005312?type=stan...,NaN
4,We'll have you working with Australia's most r...,Accounting,"Sydney, CBD, Inner West & Eastern Suburbs",Forensic Technology/Data Analytics - Senior Co...,Full Time,https://www.seek.com.au/job/38991024?type=stan...,NaN


In [35]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8354 entries, 0 to 8353
Data columns (total 7 columns):
content       8054 non-null object
industries    8054 non-null object
location      8054 non-null object
title         8054 non-null object
type_empl     8054 non-null object
url           8354 non-null object
wage          2960 non-null object
dtypes: object(7)
memory usage: 456.9+ KB


In [36]:
# Split Url out of content
def get_wage(regex, sample):
    result = re.search(regex, sample)

    if result:
#         print(result[0])
        result = result[0].strip("$").lower()
        print("B4: ",result)
        # Indicate "K" or "k" letter at the end
        if "k" in result:
            result = result.strip("k")
            result = int(result)*1000
            print("if: ",result)
            
            return result
        elif len(result) < 4:
            result = int(result)*52
            print("elif: ", result)
            
            return result
        else:
            return result

    return ''

In [37]:
frame['wage'] = frame['wage'].map(lambda row: get_wage(regex_wage,str(row)))

B4:  118
elif:  6136
B4:  700
elif:  36400
B4:  118
elif:  6136
B4:  160
elif:  8320
B4:  950
elif:  49400
B4:  650
elif:  33800
B4:  140k
if:  140000
B4:  500
elif:  26000
B4:  150k
if:  150000
B4:  100
elif:  5200
B4:  150000
B4:  110k
if:  110000
B4:  60
elif:  3120
B4:  120k
if:  120000
B4:  130k
if:  130000
B4:  200
elif:  10400
B4:  110k
if:  110000
B4:  150k
if:  150000
B4:  130k
if:  130000
B4:  130k
if:  130000
B4:  150k
if:  150000
B4:  120k
if:  120000
B4:  130000
B4:  130
elif:  6760
B4:  110k
if:  110000
B4:  120
elif:  6240
B4:  100
elif:  5200
B4:  135
elif:  7020
B4:  150k
if:  150000
B4:  800
elif:  41600
B4:  90000
B4:  160k
if:  160000
B4:  280
elif:  14560
B4:  800
elif:  41600
B4:  800
elif:  41600
B4:  850
elif:  44200
B4:  800
elif:  41600
B4:  120k
if:  120000
B4:  180k
if:  180000
B4:  1050
B4:  280
elif:  14560
B4:  550
elif:  28600
B4:  160k
if:  160000
B4:  650
elif:  33800
B4:  100k
if:  100000
B4:  105k
if:  105000
B4:  650
elif:  33800
B4:  650
elif:  338

elif:  2080
B4:  850
elif:  44200
B4:  26
elif:  1352
B4:  75
elif:  3900
B4:  55
elif:  2860
B4:  52
elif:  2704
B4:  25
elif:  1300
B4:  80k
if:  80000
B4:  29
elif:  1508
B4:  26
elif:  1352
B4:  70
elif:  3640
B4:  180k
if:  180000
B4:  70
elif:  3640
B4:  60
elif:  3120
B4:  99
elif:  5148
B4:  65k
if:  65000
B4:  75
elif:  3900
B4:  80
elif:  4160
B4:  100
elif:  5200
B4:  99
elif:  5148
B4:  65
elif:  3380
B4:  65
elif:  3380
B4:  62
elif:  3224
B4:  40
elif:  2080
B4:  50
elif:  2600
B4:  50
elif:  2600
B4:  50
elif:  2600
B4:  130
elif:  6760
B4:  150k
if:  150000
B4:  200000
B4:  130k
if:  130000
B4:  700
elif:  36400
B4:  150
elif:  7800
B4:  110k
if:  110000
B4:  600
elif:  31200
B4:  50k
if:  50000
B4:  110
elif:  5720
B4:  65
elif:  3380
B4:  100
elif:  5200
B4:  100
elif:  5200
B4:  63594
B4:  63594
B4:  113
elif:  5876
B4:  113
elif:  5876
B4:  150
elif:  7800
B4:  84
elif:  4368
B4:  84
elif:  4368
B4:  130
elif:  6760
B4:  180k
if:  180000
B4:  60
elif:  3120
B4:  67


B4:  50
elif:  2600
B4:  120
elif:  6240
B4:  153000
B4:  50
elif:  2600
B4:  90
elif:  4680
B4:  85k
if:  85000
B4:  35
elif:  1820
B4:  50
elif:  2600
B4:  30
elif:  1560
B4:  90k
if:  90000
B4:  700
elif:  36400
B4:  80k
if:  80000
B4:  90
elif:  4680
B4:  26
elif:  1352
B4:  66
elif:  3432
B4:  26
elif:  1352
B4:  200
elif:  10400
B4:  80
elif:  4160
B4:  60
elif:  3120
B4:  95
elif:  4940
B4:  100
elif:  5200
B4:  85
elif:  4420
B4:  70
elif:  3640
B4:  75k
if:  75000
B4:  85
elif:  4420
B4:  85
elif:  4420
B4:  80000
B4:  25
elif:  1300
B4:  800
elif:  41600
B4:  55
elif:  2860
B4:  42
elif:  2184
B4:  50
elif:  2600
B4:  80k
if:  80000
B4:  90
elif:  4680
B4:  80
elif:  4160
B4:  120
elif:  6240
B4:  65
elif:  3380
B4:  75
elif:  3900
B4:  30
elif:  1560
B4:  90000
B4:  75k
if:  75000
B4:  66
elif:  3432
B4:  78
elif:  4056
B4:  50
elif:  2600
B4:  100
elif:  5200
B4:  75k
if:  75000
B4:  80000
B4:  35
elif:  1820
B4:  140
elif:  7280
B4:  50k
if:  50000
B4:  80k
if:  80000
B4: 

B4:  100
elif:  5200
B4:  120k
if:  120000
B4:  90k
if:  90000
B4:  70
elif:  3640
B4:  80
elif:  4160
B4:  120
elif:  6240
B4:  70k
if:  70000
B4:  85000
B4:  80
elif:  4160
B4:  65000
B4:  55k
if:  55000
B4:  90k
if:  90000
B4:  85
elif:  4420
B4:  45
elif:  2340
B4:  65
elif:  3380
B4:  75
elif:  3900
B4:  81
elif:  4212
B4:  70
elif:  3640
B4:  90k
if:  90000
B4:  180
elif:  9360
B4:  80
elif:  4160
B4:  90k
if:  90000
B4:  120
elif:  6240
B4:  100k
if:  100000
B4:  100k
if:  100000
B4:  400
elif:  20800
B4:  80
elif:  4160
B4:  59
elif:  3068
B4:  150k
if:  150000
B4:  80
elif:  4160
B4:  130000
B4:  115k
if:  115000
B4:  120k
if:  120000
B4:  55k
if:  55000
B4:  200
elif:  10400
B4:  130k
if:  130000
B4:  150k
if:  150000
B4:  87
elif:  4524
B4:  100
elif:  5200
B4:  80
elif:  4160
B4:  100
elif:  5200
B4:  140000
B4:  150k
if:  150000
B4:  80
elif:  4160
B4:  100
elif:  5200
B4:  10k
if:  10000
B4:  65
elif:  3380
B4:  90k
if:  90000
B4:  60k
if:  60000
B4:  120k
if:  120000
B4:

In [ ]:
frame.info()

In [ ]:
frame['wage'].unique()